In [37]:
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [38]:
import xgboost

In [39]:
!pip list

Package                       Version
----------------------------- -------
alembic                       1.11.1
asttokens                     2.2.1
attrs                         23.1.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
cmaes                         0.9.1
colorama                      0.4.6
colorlog                      6.7.0
contourpy                     1.0.7
cycler                        0.11.0
debugpy                       1.5.1
decorator                     5.1.1
executing                     1.2.0
fastjsonschema                2.17.0
fonttools                     4.39.4
greenlet                      2.0.2
importlib-metadata            6.6.0
ipykernel                     6.15.0
ipython                       8.13.2
jedi                          0.18.2
joblib                        1.2.0
jsonschema                    4.17.3
jupyter_client                8.2.0
jupyter_core                  5.3.0
kiwisolver                    1.4.4
Mako           

In [40]:
df = pd.read_csv("D:\\Downloads\\Admission_Prediction.csv")

In [41]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [42]:
df.shape

(500, 9)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [44]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [45]:
df.isna().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [46]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].median())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].median())
df['University Rating'] =  df['University Rating'].fillna(df['University Rating'].median())

In [47]:
df.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [48]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [49]:
X = df.drop(["Serial No.", "Chance of Admit"], axis = 1)
y = df["Chance of Admit"]

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 42)

In [51]:
X_train.shape

(350, 7)

In [52]:
X_test.shape

(150, 7)

In [53]:
y_train.shape

(350,)

In [54]:
y_test.shape

(150,)

In [55]:
std_sca = StandardScaler()

In [56]:
X_train = std_sca.fit_transform(X_train)

In [57]:
X_train

array([[ 1.22751102,  1.29082036,  1.66979232, ..., -0.5291228 ,
         1.28550609,  0.88127734],
       [-1.64471122, -0.86291365, -0.08261341, ...,  0.01556244,
         0.07349047, -1.13471657],
       [ 0.48629237,  0.46246113, -0.08261341, ...,  0.56024767,
         0.88150088,  0.88127734],
       ...,
       [-1.36675423, -1.35992919, -1.83501915, ..., -1.61849327,
        -2.23270591, -1.13471657],
       [-0.71818792, -0.36589811, -0.95881628, ...,  0.56024767,
        -1.50886325, -1.13471657],
       [-0.25492627, -0.20022626, -0.95881628, ...,  0.01556244,
        -0.54935089, -1.13471657]])

In [58]:
X_test = std_sca.transform(X_test)

In [59]:
X_test

array([[ 1.59812034,  1.45649221,  0.79358945, ...,  0.01556244,
         1.62217709,  0.88127734],
       [-0.25492627,  0.13111743,  0.79358945, ...,  0.56024767,
         0.78049958,  0.88127734],
       [-0.16227394, -0.36589811, -0.95881628, ..., -1.07380803,
        -1.5593639 , -1.13471657],
       ...,
       [ 0.67159703,  0.95947667,  0.79358945, ...,  0.56024767,
         0.35966083, -1.13471657],
       [-0.44023093, -0.53156995, -0.08261341, ...,  0.56024767,
        -0.81868769, -1.13471657],
       [-0.44023093, -0.20022626, -0.08261341, ...,  1.64961814,
        -0.01067728, -1.13471657]])

In [ ]:
def objective(trail, data = X, target = y):
    train_X, test_X, train_y, test_y= train_test_split(data, target, test_size = .25, random_state = 42)
    param = {
        

    }